In [15]:
import pandas as pd
import os

# Get all file paths in the Nutritional_value_Data folder
paths = [
    os.path.join("raw_data", filename)
    for filename in os.listdir("raw_data")
    if os.path.isfile(os.path.join("raw_data", filename))
]

print(paths)

['raw_data/nutrient_value_some_common_foods_soups-2008-valeur_nutritive_quelques_aliments_usuels_soupe-eng.csv', 'raw_data/nutrient_value_miscellaneous_foods_2008_valeur_nutritive_divers-eng.csv', 'raw_data/nutrient_value_dairy_foods_and_other_related_products_2008_valeur_nutritive_produits_laitiers_et_produits_annexes-eng.csv', 'raw_data/nutrient_value_legumes_nuts_and_seeds_2008_valeur_nutritive_legumineuses_noix_et_graines-eng.csv', 'raw_data/nutrient_value_meat_and_poultry_2008_valeur_nutritive_2008_viande_et_volaille-eng.csv', 'raw_data/nutrient_value_vegetables_and_vegetable_products_2008_valeur_nutritive_legumes_et_produits_base_de_legumes-eng.csv', 'raw_data/nutrient_value_some_common_foods_sweets_sugars-2008-valeur_nutritive_quelques_aliments_usuels_sucres_sucreries-eng.csv', 'raw_data/nutrient_value_mixed_dishes_2008_valeur_nutritive_mets_composes-eng.csv', 'raw_data/nutrient_value_fats_and_oils_foods_2008_valeur_nutritive_matieres_grasses_et_huiles-eng.csv', 'raw_data/nutrie

In [16]:
# Leave only columns that are present in all the files
# The data is not very clean, and usually there are two cases how to clearn the data
cols_all = set()

for path in paths:
    df = pd.read_csv(path, on_bad_lines='skip', encoding='ISO-8859-1')
    df = df.dropna(how='all')
    if df.iloc[1, 0] == 'Food name':
        df = df.iloc[1:, :]
        df.columns = df.iloc[0]
        df = df[1:].reset_index(drop=True)
        category = df.iloc[0, 0]
        df = df.iloc[1:, :]
        df['Food Category'] = category
        df = df.loc[:, df.columns.notna()]
        cols = set(df.columns)
        cols_all = cols_all.union(cols)
    elif df.iloc[2, 0] == 'Food name':
        df = df.iloc[1:, :]
        category = df.iloc[0, 0]
        df = df.iloc[1:, :]
        df.columns = df.iloc[0]
        df = df[3:].reset_index(drop=True)
        df['Food Category'] = category
        df = df.loc[:, df.columns.notna()]
        cols = set(df.columns)
        cols_all = cols_all.union(cols)
common_cols = list(cols_all)
common_cols.insert(0, common_cols.pop(common_cols.index('Food Category')))
common_cols.insert(1, common_cols.pop(common_cols.index('Food name')))
common_cols.insert(2, common_cols.pop(common_cols.index('Weight ')))
common_cols.insert(3, common_cols.pop(common_cols.index('Measure')))
common_cols.insert(4, common_cols.pop(common_cols.index('Protein')))
common_cols.insert(5, common_cols.pop(common_cols.index('Carbohydrate')))
common_cols.insert(6, common_cols.pop(common_cols.index('Total Fat')))
common_cols

['Food Category',
 'Food name',
 'Weight ',
 'Measure',
 'Protein',
 'Carbohydrate',
 'Total Fat',
 'Vitamin D',
 'Energy',
 'Phosphorus',
 'Folate ',
 'Cholesterol',
 'Magnesium',
 'Vitamin B12',
 'Calcium',
 'DHA',
 'Monounsaturated fat',
 'Vitamin C',
 'Monounsaturated Fat',
 'Thiamin',
 'Trans Fat',
 'Sodium',
 'Vitamin A',
 'Riboflavin',
 'Folate',
 'Trans fats',
 'Alcohol',
 'Protei',
 'Total Dietary Fibre',
 'Polyunsaturated Fat',
 'EPA',
 'Niacin',
 'Saturated Fat',
 'Polyunsaturated fat',
 'Vitamin E',
 'Total Sugar',
 'Iron',
 'Beta carotene',
 'Lycopene',
 ' Folate',
 'Caffeine',
 'Dietary Folate Equivalents',
 'Potassium',
 'Total Daietary Fibre']

In [17]:
# Combine all the individual files into one big dataframe
# The data is not very clean, and usually there are two cases how to clearn the data
big_df = pd.DataFrame()
problematic_paths = []
for path in paths:
    try:
        df = pd.read_csv(path, on_bad_lines='skip', encoding='ISO-8859-1')
        df = df.dropna(how='all')
        if df.iloc[1, 0] == 'Food name':
            print('good (case 1)')
            df = df.iloc[1:, :]
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)
            category = df.iloc[0, 0]
            df = df.iloc[1:, :]
            df['Food Category'] = category
            df = df.loc[:, df.columns.notna()]
            cols = df.columns.tolist()
            for col in common_cols:
                if col not in cols:
                    df[col] = 'N/A'
            df = df[common_cols]
            big_df = pd.concat([big_df, df])
        elif df.iloc[2, 0] == 'Food name':
            print('good (case 2)')
            df = df.iloc[1:, :]
            category = df.iloc[0, 0]
            df = df.iloc[1:, :]
            df.columns = df.iloc[0]
            df = df[3:].reset_index(drop=True)
            df['Food Category'] = category
            df = df.loc[:, df.columns.notna()]
            cols = df.columns.tolist()
            for col in common_cols:
                if col not in cols:
                    df[col] = 'N/A'
            df = df[common_cols]
            big_df = pd.concat([big_df, df])
        else:
            print('error')
            problematic_paths.append(path)
    except Exception as e:
        print(str(e))
        problematic_paths.append(path)

good (case 2)
good (case 2)
good (case 1)
good (case 2)
good (case 2)
good (case 1)
good (case 2)
good (case 2)
good (case 2)
good (case 1)
good (case 1)
good (case 2)
good (case 1)
good (case 2)
good (case 1)
good (case 2)
good (case 2)


In [18]:
big_df

,Food Category,Food name,Weight,Measure,Protein,Carbohydrate,Total Fat,Vitamin D,Energy,Energy,...,Vitamin E,Total Sugar,Iron,Beta carotene,Lycopene,Folate,Caffeine,Dietary Folate Equivalents,Potassium,Total Daietary Fibre
0,SOUPS,"Beef or chicken, broth/bouillon",254,250mL,3,tr,1,N/A,18,74,...,N/A,0,0.4,N/A,N/A,N/A,N/A,N/A,137,N/A
1,SOUPS,"Beef, chunky",254,250mL,12,21,5,N/A,180,753,...,N/A,2,2.5,N/A,N/A,N/A,N/A,N/A,355,N/A
2,SOUPS,"Chicken noodle, chunky",254,250mL,13,18,6,N/A,185,773,...,N/A,2,1.5,N/A,N/A,N/A,N/A,N/A,114,N/A
3,SOUPS,"Chicken noodle, low fat, reduced salt",257,250mL,9,13,2,N/A,105,444,...,N/A,1,0,N/A,N/A,N/A,N/A,N/A,NaN,N/A
4,SOUPS,"Chicken vegetable, chunky",254,250mL,13,20,5,N/A,175,733,...,N/A,NaN,1.5,N/A,N/A,N/A,N/A,N/A,388,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,SNACKS,"Rice cakes, plain",9,1,1,7,tr,N/A,35,146,...,N/A,tr,0.1,N/A,N/A,N/A,N/A,N/A,26,N/A
21,SNACKS,"Sesame sticks, salted",14,60mL,2,6,5,N/A,75,316,...,N/A,tr,0.1,N/A,N/A,N/A,N/A,N/A,25,N/A
22,SNACKS,"Soybeans, roasted, salted",44,60mL,15,15,11,N/A,205,859,...,N/A,2,1.7,N/A,N/A,N/A,N/A,N/A,641,N/A
23,SNACKS,"Trail mix, regular",38,60mL,5,17,11,N/A,176,735,...,N/A,NaN,1.2,N/A,N/A,N/A,N/A,N/A,261,N/A


In [19]:
big_df.to_csv('Food_all.csv', index=False)